In [1]:
import os
from typing import Literal
from operator import itemgetter
from typing_extensions import TypedDict
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

## Third-party data

source data -> text loader -> embedding vector -> store vectors -> retrieve vectors

In [3]:
# # TODO: add third-party data
# loader = TextLoader("./data/recipes.json")
# index = VectorstoreIndexCreator(embedding=OpenAIEmbeddings()).from_loaders([loader]) 

# data = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# # k is the number of chunks to retrieve
# k = 4
# retriever = vectorstore.as_retriever(k=k)

In [4]:
# query = "How to make Classic Margherita Pizza?"
# result = index.query_with_sources(query, llm=OpenAI())

## Build chains for different use cases

In [6]:
def chain_creator(llm, prompt_template):
    return prompt_template | llm | StrOutputParser()

def create_prompt_template(system_message: str) -> ChatPromptTemplate:
    return ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{query}"),
        ]
    )

In [7]:
# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Define system messages for each service
service_prompts = {
    "tourist_attraction": "You are an expert in finding tourist attractions.",
    "itinerary_planning": "You are an expert in travel itinerary planning.",
    "restaurant_recommendations": "You are an expert in recommending restaurants.",
    "exploring_travel_ideas": "You are an expert in recommending travel ideas.",
    "others": "You are a friendly and helpful chatbot."
}

# Create chains for each service
service_chains = {service: chain_creator(llm=llm, prompt_template=create_prompt_template(service_prompts[service])) for service in service_prompts.keys()}

## Route an input query to a specific chain 

In [9]:
# Define the routing system
class RouteQuery(TypedDict):
    """Route query to the appropriate service_type."""
    service_type: Literal[
        "tourist_attraction", 
        "itinerary_planning", 
        "restaurant_recommendations", 
        "exploring_travel_ideas",
        "others"
    ]

route_system = "Route the user's query to either 'tourist_attraction', 'itinerary_planning', 'restaurant_recommendations', 'exploring_travel_ideas', or 'others' if it doesn't fit into the previous categories."

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)

# Routing chain based on the query
route_chain = route_prompt | llm.with_structured_output(RouteQuery) | itemgetter("service_type")

# Final chain that routes to the appropriate LLM chain based on user input
def get_chain_for_service(service_type, query):
    print("service_type: ", service_type)
    print("query: ", query)
    if service_type in service_chains.keys():
        return service_chains.get(service_type).invoke({"query": query})
    else:
        raise ValueError(f"Unknown service type: {service_type}")
    
# Combine all the steps into the final chain
chatbot_chain = {
    "service_type": route_chain,
    "query": lambda x: x['query']
} | RunnableLambda(
    # Final step: Route to the correct service and invoke the right chain
    lambda x: get_chain_for_service(x["service_type"], x['query'])
)

## Start

In [12]:
if __name__ == "__main__":

    inout_query = {
        "query": "Could you recommend 3 delicious and cheap restaurants in Victoria BC?"
    }    
    response = chatbot_chain.invoke(inout_query)
    print(f"Response: {response}")

service_type:  restaurant_recommendations
query:  Could you recommend 3 delicious and cheap restaurants in Victoria BC?
Response: Of course! Here are three delicious and affordable restaurants in Victoria, BC:

1. Red Fish Blue Fish - Located in a converted cargo container on the waterfront, Red Fish Blue Fish serves up delicious and sustainable seafood dishes at affordable prices. Their fish tacos and seafood poutine are highly recommended.

2. Foo Asian Street Food - This casual eatery in downtown Victoria offers a variety of Asian street food dishes at budget-friendly prices. Their bao buns, rice bowls, and noodle dishes are sure to satisfy your cravings without breaking the bank.

3. La Taquisa - For tasty and affordable Mexican cuisine, head to La Taquisa. With multiple locations in Victoria, this restaurant offers a menu full of traditional Mexican dishes like tacos, burritos, and quesadillas made with fresh and flavorful ingredients.

These restaurants are known for their delici